In [4]:
# ================================
# SIMPLE: Margin % by Category
# ================================
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# -----------------------------
# CONFIGURATION
# -----------------------------
CONFIG = {
    'sql_query': """
        SELECT 
            CategoryName,
            NetSales AS Revenue,
            [GM$] AS GrossMarginDollar
        FROM _Gift_from_Carpenters 
        WHERE NetSales IS NOT NULL
        AND [GM$] IS NOT NULL
        AND NetSales > 0
        AND CategoryName IS NOT NULL
    """,
    'server': '100.99.225.51',
    'database': 'TEST1',
    'username': 'SA',
    'password': 'a31536000',
    'driver': '{ODBC Driver 17 for SQL Server}',
    'target_table': 'category_margin_analysis'
}

# -----------------------------
# 1. LOAD DATA
# -----------------------------
def load_data():
    conn_str = (
        f"DRIVER={CONFIG['driver']};"
        f"SERVER={CONFIG['server']};"
        f"DATABASE={CONFIG['database']};"
        f"UID={CONFIG['username']};"
        f"PWD={CONFIG['password']};"
        f"Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=300"
    )
    print("Connecting to database...")
    conn = pyodbc.connect(conn_str)
    df = pd.read_sql(CONFIG['sql_query'], conn)
    conn.close()
    print(f"Loaded {len(df):,} rows")
    return df

# -----------------------------
# 2. CALCULATE MARGIN %
# -----------------------------
def calculate_margin(df):
    result = df.groupby('CategoryName').agg(
        TotalRevenue=('Revenue', 'sum'),
        TotalProfit=('GrossMarginDollar', 'sum')
    ).reset_index()

    result['MarginPct'] = (result['TotalProfit'] / result['TotalRevenue'] * 100).round(2)
    result['RunDate'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    result = result[['CategoryName', 'TotalRevenue', 'TotalProfit', 'MarginPct', 'RunDate']]
    
    print(f"Calculated margin for {len(result)} categories")
    return result

# -----------------------------
# 3. SAVE TO SQL 
# -----------------------------
def save_to_sql(df):
    engine = create_engine(
        f"mssql+pyodbc://{CONFIG['username']}:{CONFIG['password']}@"
        f"{CONFIG['server']}/{CONFIG['database']}?"
        f"driver=ODBC+Driver+17+for+SQL+Server&Encrypt=yes&TrustServerCertificate=yes"
    )
    table_name = CONFIG['target_table']
    try:
        df.to_sql(table_name, engine, if_exists='replace', index=False, schema='dbo')
        print(f"Table 'dbo.{table_name}' saved to database")
        
        # CONFIRM IT'S THERE
        count_df = pd.read_sql(f"SELECT COUNT(*) AS cnt FROM dbo.{table_name}", engine)
        row_count = count_df['cnt'].iloc[0]
        print(f"Verified: {row_count} rows in table")
        
    except Exception as e:
        print(f"SAVE FAILED: {e}")

# -----------------------------
# MAIN
# -----------------------------
def main():
    print("Starting Margin % by Category...\n")
    df = load_data()
    result = calculate_margin(df)
    save_to_sql(result)
    
    print("\n" + "="*50)
    print("MARGIN ANALYSIS COMPLETE!")
    print("="*50)
    print("Table: category_margin_analysis")
    print("Key Column: MarginPct (%)")
    print("Ready for Power BI, Excel, filtering!")
    print("="*50)

if __name__ == "__main__":
    main()

Starting Margin % by Category...

Connecting to database...
Loaded 27,152 rows
Calculated margin for 221 categories
Table 'dbo.category_margin_analysis' saved to database
Verified: 221 rows in table

MARGIN ANALYSIS COMPLETE!
Table: category_margin_analysis
Key Column: MarginPct (%)
Ready for Power BI, Excel, filtering!
